# AuxTel accelerometer PSD - 29-Sep-22
Diagnosing bad accelerometer channel \
Craig Lage

In [ ]:
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client.efd_helper import EfdClient

In [ ]:
# Get EFD client
client = EfdClient('summit_efd')

In [ ]:
# Just using this in the notebook for now.
def merge_packed_PSD(packed_dataframe, base_field, sensor_names):
    """Select fields that represent the Power Spectral Density of \                                                          
    a sensor and unpack them into a dataframe with PSD vs frequency.                                                         
    Parameters                                                                                                               
    ----------                                                                                                               
    packed_dataframe : `pandas.DataFrame`                                                                                    
        packed data frame containing the desired data                                                                        
    base_field :  `str`                                                                                                      
        Base field name that will be expanded to query all                                                                   
        vector entries.                                                                                                      
    sensor_name :  `str` or list                                                                                             
        Name of the sensor(s) of interest.                                                                                   
    Returns                                                                                                                  
    -------                                                                                                                  
    result : `pandas.DataFrame`                                                                                              
        A `pandas.DataFrame` containing the results of the query.                                                            
    """
    minPSDFrequency = packed_dataframe['minPSDFrequency'][0]
    maxPSDFrequency = packed_dataframe['maxPSDFrequency'][0]
    numDataPoints = packed_dataframe['numDataPoints'][0]
    if isinstance(sensor_names, str):
        sensor_names = [sensor_names, ]

    packed_dataframe = packed_dataframe.loc[packed_dataframe.sensorName.isin(sensor_names)]
    packed_fields = [k for k in packed_dataframe.keys()
                     if k.startswith(base_field) and k[len(base_field):].isdigit()]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID                         
    npack = len(packed_fields)
    assert numDataPoints == npack, "Number of packed data points does not match numDataPoints!"
    packed_len = len(packed_dataframe)
    output = np.empty(npack * packed_len)
    deltaF = float(maxPSDFrequency - minPSDFrequency) / (npack - 1) # Frequency step                                         
    columns = []
    for i in range(npack):
        label = f"{base_field}{i}"
        columns.append(minPSDFrequency + i * deltaF)
        output[i::npack] = packed_dataframe[label]
    output = np.reshape(output, (packed_len, npack))
    return pd.DataFrame(data=output, columns=columns, index=packed_dataframe.index)

# This dictionary defines the axis scramble
trueAxes = {'AuxTel-M1':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-M2':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-Truss':{'X':'El', 'Y':'Opt', 'Z':'Az'}}

In [ ]:
# Times to start looking at PSD data
start = Time("2022-10-07 12:00:30Z", scale='utc')
end = Time("2022-10-07 12:00:40Z", scale='utc')

# First, verify problem is still there

In [ ]:
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)

indexCounter = 0
plt.figure(figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.3)
axes = ['X', 'Y', 'Z']
sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
plotCounter = 1
for sensor in sensors:
    for axis in axes:
        base_field = f"accelerationPSD{axis}"
        plt.subplot(3,3,plotCounter)
        plt.title(f"{sensor} - {axis}", fontsize=12)
        df = merge_packed_PSD(accel_data, base_field, sensor)
        row = df.iloc[indexCounter][2:]
        row.plot()
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD [m^2/(Hz s^4)]')
        plotCounter += 1
timestamp = df.index[0].strftime("%Y%m%dT%H%M%SZ")
plt.suptitle(f"Accelerometer Power Spectral Density - {timestamp}", fontsize=16)
#plt.savefig(f"/home/craiglagegit/DATA/Accel_PSD_{timestamp}.pdf")

In [ ]:
# Times to start looking at PSD data
start = Time("2022-09-29 13:31:30Z", scale='utc')
end = Time("2022-09-29 13:31:40Z", scale='utc')

# Now see if it's fixed - Yes!

In [ ]:
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)

indexCounter = 0
plt.figure(figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.3)
M12_axes = ['X', 'Y', 'Z']
truss_axes = ['X', 'Z', 'Y']
sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
plotCounter = 1
for sensor in sensors:
    if sensor in ["AuxTel-M1", "AuxTel-M2"]:
        axes = M12_axes
    else:
        axes = truss_axes
    for axis in axes:
        base_field = f"accelerationPSD{axis}"
        trueAxis = trueAxes[sensor][axis]
        plt.subplot(3,3,plotCounter)
        plt.title(f"{sensor} - {trueAxis}", fontsize=12)
        df = merge_packed_PSD(accel_data, base_field, sensor)
        row = df.iloc[indexCounter][2:]
        row.plot()
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD [m^2/(Hz s^4)]')
        plotCounter += 1
timestamp = df.index[0].strftime("%Y%m%dT%H%M%SZ")
plt.suptitle(f"Accelerometer Power Spectral Density - {timestamp}", fontsize=16)
plt.savefig(f"/home/craiglagegit/DATA/Accel_PSD_{timestamp}.pdf")

In [ ]:
#Re-plot on same scale
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)

indexCounter = 0
plt.figure(figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.3)
M12_axes = ['X', 'Y', 'Z']
truss_axes = ['X', 'Z', 'Y']
sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
plotCounter = 1
for sensor in sensors:
    if sensor in ["AuxTel-M1", "AuxTel-M2"]:
        axes = M12_axes
    else:
        axes = truss_axes
    for axis in axes:
        base_field = f"accelerationPSD{axis}"
        trueAxis = trueAxes[sensor][axis]
        plt.subplot(3,3,plotCounter)
        plt.title(f"{sensor} - {trueAxis}", fontsize=12)
        df = merge_packed_PSD(accel_data, base_field, sensor)
        row = df.iloc[indexCounter][2:]
        row.plot()
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD [m^2/(Hz s^4)]')
        plt.ylim(0.0, 5.0E-10)
        plotCounter += 1
timestamp = df.index[0].strftime("%Y%m%dT%H%M%SZ")
plt.suptitle(f"Accelerometer Power Spectral Density - {timestamp}", fontsize=16)
plt.savefig(f"/home/craiglagegit/DATA/Accel_PSD_Scaled_{timestamp}.pdf")

In [ ]:
# Basic plot
# Times to start looking at PSD data
start = Time("2022-09-30 00:16:52Z", scale='utc')
end = Time("2022-09-30 00:16:56Z", scale='utc')
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)

indexCounter = 0
plt.figure(figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.3)
M12_axes = ['X', 'Y', 'Z']
truss_axes = ['X', 'Z', 'Y']
sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
plotCounter = 1
for sensor in sensors:
    if sensor in ["AuxTel-M1", "AuxTel-M2"]:
        axes = M12_axes
    else:
        axes = truss_axes
    for axis in axes:
        base_field = f"accelerationPSD{axis}"
        trueAxis = trueAxes[sensor][axis]
        plt.subplot(3,3,plotCounter)
        plt.title(f"{sensor} - {trueAxis}", fontsize=12)
        df = merge_packed_PSD(accel_data, base_field, sensor)
        row = df.iloc[indexCounter][2:]
        row.plot()
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD [m^2/(Hz s^4)]')
        plt.ylim(0.0, 5.0E-10)
        plotCounter += 1
timestamp = df.index[0].strftime("%Y%m%dT%H%M%SZ")
plt.suptitle(f"Accelerometer Power Spectral Density - {timestamp}", fontsize=16)
plt.savefig(f"/home/craiglagegit/DATA/Accel_PSD_Scaled_{timestamp}.pdf")